# Modelo Megaline

Prueba de modelos para analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Para alcanzar el objetivo propuesto por Megaline nos enfrentamos a una tarea de aprendizaje supervisado, específicamente una tarea de **clasificación binaria** al tener que encontrar entre dos planes, el mejor para los clientes. 

In [1]:
#Librerías utilizadas.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('./users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
print(df.shape)

(3214, 5)


### Observaciones a partir del data frame:

En este dataframe contamos con cinco características y 3214 observaciones. El objetivo a identificar con el modelo de Machine learning es **Is Ultra**.

Además, podemos observar que no hay datos ausentes y todas las columnas se encuentran configuradas en el tipo adecuado. 

Al no existir el conjunto de prueba, todo el dataset se dividirá en una proporción de 3:1:1 = entrenamiento(60 %), validación (20 %) y prueba (20 %).

## Segmentación de datos

In [6]:
df_train, df_test = train_test_split(df, test_size = 0.4, train_size = 0.6, random_state = 333)


In [7]:
df_test, df_valid = train_test_split(df_test, test_size = 0.5, train_size = 0.5, random_state = 333)

In [8]:
#Datos para el entrenamiento.
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

#Datos para el test.
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

#Datos para la validación.
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

In [9]:
print(train_features.shape)
print(train_target.shape)
print()
print(test_features.shape)
print(test_target.shape)
print()
print(valid_features.shape)
print(valid_target.shape)

(1928, 4)
(1928,)

(643, 4)
(643,)

(643, 4)
(643,)


## Prueba 1: 
## Árbol de dicisión

In [10]:
#Iteración para encontrar qué nivel de ramificación hace más exacto el modelo. 

best_acc_dt = 0

best_depth_dt = 0

for depth in range(1, 11):
    model_dt = DecisionTreeClassifier(random_state = 333, max_depth = depth)
    model_dt.fit(train_features, train_target)
    valid_predictions = model_dt.predict(valid_features)
    acc = accuracy_score(valid_target, valid_predictions)
    if acc > best_acc_dt:
        best_acc_dt = acc
        best_depth_dt = depth

print()
print(f'The best model found has a depth of {best_depth_dt} with an accuracy of {best_acc_dt}')


The best model found has a depth of 5 with an accuracy of 0.7900466562986003


In [11]:
#Prueba para ver que tanta exactitud posee el modelo sin limitar las ramificaciones. 

model_dt = DecisionTreeClassifier(random_state = 333)
    
model_dt.fit(train_features, train_target)

valid_predictions = model_dt.predict(valid_features)

print(accuracy_score(valid_target, valid_predictions))

0.7122861586314152


In [12]:
#Prueba con hiperparámetro: min_samples_split.

best_acc_dt_split = 0

best_split_dt = 0

for split in range(2, 11):
    model_dt = DecisionTreeClassifier(random_state = 333, min_samples_split = split)
    model_dt.fit(train_features, train_target)
    valid_predictions = model_dt.predict(valid_features)
    acc_split = accuracy_score(valid_target, valid_predictions)
    if acc_split > best_acc_dt_split:
        best_acc_dt_split = acc_split
        best_split_dt = split
        
print()
print(f'The best model found has a number of splits of {best_split_dt} with an accuracy of {best_acc_dt_split}')


The best model found has a number of splits of 9 with an accuracy of 0.7356143079315708


Al ser una exactitud menor que la que se logra con el parámetro de número de ramificaciones, se ignorará el parámetro de 'split' durante la prueba del modelo final.

In [13]:
#Prueba de exactitud con el modelo elegido en su versión test y entrenamiento:

model_dt = DecisionTreeClassifier(random_state = 333, max_depth = 5)

model_dt.fit(train_features, train_target)

train_predictions = model_dt.predict(train_features)

test_predictions = model_dt.predict(test_features)

print('Accuracy comparison: Decision Tree')
print()
print('Training set:', accuracy_score(train_target, train_predictions))
print('Test set:', accuracy_score(test_target, test_predictions))

Accuracy comparison: Decision Tree

Training set: 0.8236514522821576
Test set: 0.7993779160186625


Según la prueba de exactitud, nos encontramos con un modelo un tanto sobreajustado, al tener un mejor resultado durante el entrenamiento y un resultado menor durante la prueba. 

In [14]:
#Exactitud del modelo Árbol de decisión.

valid_predictions = model_dt.predict(valid_features)

acc_valid_dt = accuracy_score(valid_target, valid_predictions)

print('Accuracy Decision Tree')
print()
print('The accuracy of the Decision tree model in its validation set is:', acc_valid_dt)

Accuracy Decision Tree

The accuracy of the Decision tree model in its validation set is: 0.7900466562986003


#### Observaciones: 

El modelo de árbol de decisión pasa el umbral mínimo de exactitud del 0.75 por tan solo 0.04 puntos.  

## Prueba 2: 
## Bosque aleatorio

In [15]:
#Prueba para elegir el mejor hiperparámetro para el modelo de Random Forest.

best_score = 0

best_est = 0

for est in range(1, 50): 
    model_rf = RandomForestClassifier(random_state = 333, n_estimators=est) 
    model_rf.fit(train_features, train_target) 
    score = model_rf.score(valid_features, valid_target)
    if score > best_score:
        best_score = score
        best_est = est

print("The accuracy of the best model in the validation set(n_estimators = {}): {}".format(best_est, best_score))

The accuracy of the best model in the validation set(n_estimators = 38): 0.8087091757387247


In [16]:
#Modelo con los hiperparámetros seleccionados.

model_rf = RandomForestClassifier(random_state = 333, n_estimators = 38) 

model_rf.fit(train_features, train_target) 

acc_train_rf = model_rf.score(train_features, train_target)

acc_test_rf = model_rf.score(test_features, test_target)

print('Accuracy comparison: Random Forest')
print()
print('Training set:', acc_train_rf)
print('Test set:', acc_test_rf)

Accuracy comparison: Random Forest

Training set: 0.9968879668049793
Test set: 0.7853810264385692


#### Observaciones: 

Se puede observar que la exactitud del modelo es casi perfecta durante el entrenamiento, sin embargo durante el test baja al menos 0.21 puntos. 

In [17]:
#Exactitud del modelo Bosque aleatorio.

acc_valid_rf = model_rf.score(valid_features, valid_target)

print('The accuracy of the Random forest model in its validation set is:', acc_valid_rf)

The accuracy of the Random forest model in its validation set is: 0.8087091757387247


#### Observaciones: 

Al validar la exactitud del modelo de Bosque aleatorio podemos ver que este supera el umbral requerido en al menos 0.05 puntos, siendo el mejor modelo hasta ahora. 

## Prueba 3: 
## Regresión logística

In [18]:
#Implementación del modelo de Regresión logística.

model_lr = LogisticRegression(random_state = 333, solver = 'liblinear')

model_lr.fit(train_features, train_target)

score_train = model_lr.score(train_features, train_target) 

score_test = model_lr.score(test_features, test_target) 

print('Accuracy comparison: Logistic Regression')
print()
print('Training set:', score_train)
print('Test set:', score_test)

Accuracy comparison: Logistic Regression

Training set: 0.7090248962655602
Test set: 0.6547433903576982


In [19]:
#Exactitud del modelo Regresión logística.

acc_valid_lr = model_lr.score(valid_features, valid_target)

print('The accuracy of the Logistic regression model in its validation set is:', acc_valid_lr)

The accuracy of the Logistic regression model in its validation set is: 0.6967340590979783


#### Observaciones: 

El modelo elaborado con Regresión logística obtuvo los peores resultados de exactitud, donde ni siquiera la prueba durante el entrenamiento superó el umbral requerido de 0.75. 

## Comparación entre modelos

In [20]:
print('Decision tree model accuracy:', acc_valid_dt.round(3))
print('Random forest model accuracy:', acc_valid_rf.round(3))
print('Logistic regression model accuracy:', acc_valid_lr.round(3))

Decision tree model accuracy: 0.79
Random forest model accuracy: 0.809
Logistic regression model accuracy: 0.697


Comparando los números podemos determinar que el mejor modelo, en cuanto a exactitud, es el de Bosque aleatorio.

## Prueba de cordura en el modelo seleccionado

Para hacer la prueba de cordura al modelo para buscar problemas de clasificación necesitamos comparar sus predicciones con posibilidad aleatoria, esto se hará por medio de un Dummy Classifier que ignore las caracteríticas del modelo y fije un valor predeterminado a todos los usuarios en la columna 'Is Ultra'.

In [21]:
ultra_users = df[df['is_ultra'] == 1]

non_ultra_users = df[df['is_ultra'] == 0]

print('Users that have an Ultra plan:', ultra_users['is_ultra'].count())
print('Users that does not have an Ultra plan:', non_ultra_users['is_ultra'].count())

Users that have an Ultra plan: 985
Users that does not have an Ultra plan: 2229


Estas cantidades nos permiten seleccionar un mejor parámetro para hacer un Dummy Classifier y así lograr la prueba de cordura. 

En este caso, los parámetros 'stratified' o 'most frequent' pueden ser los adecuados cuando nos enfrentamos a una clasificación más dominante o un resultado más abundante en los datos, como lo son los usuarios que no poseen un plan Ultra.

In [22]:
#Ejecución del Dummy Classifier para hacer una prueba de cordura. 

dummy_clf = DummyClassifier(random_state = 333, strategy = 'most_frequent')

dummy_clf.fit(train_features, train_target)

acc_dummy_clf = dummy_clf.score(valid_features, valid_target)

print(acc_dummy_clf)

0.6936236391912908


#### Observaciones:

Esta ejecución del Dummy Classifier logró que todos los resultados de la columna 'Is Ultra' fueran igual a 0, que es el dato dominante, tras ello se hace una prueba de exactitud que determinó que este modelo logra el resultado correcto el 69 % de las veces. 

In [23]:
#Comparación de exactitud entre el modelo Random Forest y Dummy Classifier.

print("Accuracy Most Frequent Class Dummy Classifier:", acc_dummy_clf)
print()
print("Accuracy Random Forest:", acc_valid_rf)

Accuracy Most Frequent Class Dummy Classifier: 0.6936236391912908

Accuracy Random Forest: 0.8087091757387247


## Conclusiones

1. El mejor modelo para lograr superar el umbral de exactitud de 0.75 es el Bosque aleatorio con el hiperparámetro de n_estimators = 38.

2. El segundo mejor modelo es el Árbol de decisión, que también pasa el umbral requerido.

3. El único modelo que no supera el umbral y se descarta es el de Regresión logística.

4. Se hizo una prueba de cordura con un modelo tipo Dummy Classifier, que ignora parámetros y da una respuesta a la columna objetivo con el dato dominante. 

5. Tras efectuar la prueba y comparar la exactitud del modelo de Bosque aleatorio y el Dummy Classifier se puede determinar que el modelo elaborado supera la exactitud de la prueba con al menos 0.11 puntos, convirtiéndose en la mejor alternativa para Megaline. 